# Pyro Examples

In [1]:
import pandas as pd
import numpy as np
from orbit.lgt import LGT
from orbit.utils.plot import plot_predicted_data
from orbit.utils.plot import plot_predicted_components

In [2]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

## Data

In [3]:
DATA_FILE = "./data/iclaims_example.csv"

In [4]:
raw_df = pd.read_csv(DATA_FILE, parse_dates=['week'])

### Train / Test Split

In [5]:
df=raw_df.copy()
test_size=52
train_df=df[:-test_size]
test_df=df[-test_size:]

In [6]:
train_df.head(5)

,week,claims,trend.unemploy,trend.filling,trend.job
0,2010-01-03,651215,1.18397,0.72014,1.11967
1,2010-01-10,825891,1.18397,0.81490,1.17860
2,2010-01-17,659173,1.20338,0.73909,1.11967
3,2010-01-24,507651,1.16456,0.81490,1.10788
4,2010-01-31,538617,1.08693,0.77699,1.07253


## MAP Fit and Predict

In [7]:
lgt_map=LGT(
    response_col="claims",
    date_col="week",
    seasonality=52,
    seed=8888,
    inference_engine='pyro',
    sample_method='map',
    predict_method='map',
    auto_scale=False,
    is_multiplicative=True,
    verbose=True,
)

Using 4 chains, 8 cores, 225 warmup and 25 samples per chain for sampling.


In [8]:
lgt_map.fit(df=train_df)

RuntimeError: invalid argument 0: Tensors must have same number of dimensions: got 1 and 2 at ../aten/src/TH/generic/THTensor.cpp:603
 Trace Shapes:     
  Param Sites:     
 Sample Sites:     
   lev_sm dist |   
         value |   
   slp_sm dist |   
         value |   
       nu dist |   
         value |   
obs_sigma dist |   
         value |   
  lt_coef dist |   
         value |   
  gt_coef dist |   
         value |   
   gt_pow dist |   
         value |   
   sea_sm dist |   
         value |   
 init_sea dist | 52
         value | 52
Trace Shapes:
 Param Sites:
Sample Sites:

In [ ]:
predicted_df = lgt_map.predict(df=test_df)

In [ ]:
plot_predicted_data(training_actual_df=train_df, predicted_df=predicted_df, 
                    date_col=lgt_map.date_col, actual_col=lgt_map.response_col, pred_col='prediction', 
                    test_actual_df=test_df)

## VI Fit and Predict

Pyro only support Variational Inference(SVI) for full sampling prediction.

In [ ]:
lgt_vi = LGT(
    response_col='claims',
    date_col='week',
    seasonality=52,
    num_warmup=4000, 
    num_sample=500,
    seed=8888,
    inference_engine='pyro',
    sample_method='vi',
    predict_method='full',
    n_bootstrap_draws=500,
    verbose=True,
)

In [ ]:
lgt_vi.fit(df=train_df)

In [ ]:
predicted_df = lgt_vi.predict(df=test_df)

In [ ]:
plot_predicted_data(training_actual_df=train_df, predicted_df=predicted_df, 
                    date_col=lgt_vi.date_col, actual_col=lgt_vi.response_col, pred_col=50, 
                    pred_quantiles_col=[5,95], test_actual_df=test_df)

## Fit with Regressors

In [ ]:
lgt_reg_map=LGT(
    response_col='claims',
    regressor_col=['trend.unemploy', 'trend.filling'],
    regressor_sign=['=','='],
    date_col='week',
    seasonality=52,
    seed=8888,
    inference_engine='pyro',
    sample_method='vi',
    predict_method='full'
)

In [ ]:
lgt_reg_map.fit(df=train_df)

In [ ]:
predicted_df = lgt_reg_map.predict(df=test_df)

In [ ]:
predicted_df.head(5)

In [ ]:
plot_predicted_data(training_actual_df=train_df, predicted_df=predicted_df, 
                    date_col=lgt_reg_map.date_col, actual_col=lgt_reg_map.response_col, pred_col='prediction', 
                    test_actual_df=test_df)

In [ ]:
predicted_df = lgt_reg_map.predict(df=df, decompose=True)
predicted_df.head(5)

In [ ]:
plot_predicted_components(predicted_df=predicted_df, date_col='week')